In [1]:
%matplotlib inline

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import AdamOptimizer, GradientDescentOptimizer
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import math
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

np.random.seed(42)


def density_matrix(state):
    return state * np.conj(state).T


label_0 = [[1],[0],[0],[0],[0],[0],[0],[0],[0],[1],[0],[0],[0],[0],[0],[0]]
label_1 = [[0],[1],[0],[0],[0],[0],[0],[0],[0],[0],[1],[0],[0],[0],[0],[0]]
label_2 = [[0],[0],[1],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]]
label_3 = [[0],[0],[0],[1],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]]
label_4 = [[0],[0],[0],[0],[1],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]]
label_5 = [[0],[0],[0],[0],[0],[1],[0],[0],[0],[0],[0],[0],[0],[0],[0],[0]]
label_6 = [[0],[0],[0],[0],[0],[0],[1],[0],[0],[0],[0],[0],[0],[0],[0],[0]]
label_7 = [[0],[0],[0],[0],[0],[0],[0],[1],[0],[0],[0],[0],[0],[0],[0],[0]]
label_8 = [[0],[0],[0],[0],[0],[0],[0],[0],[1],[0],[0],[0],[0],[0],[0],[0]]

state_labels = [label_0, label_1, label_2, label_3, label_4, label_5, label_6, label_7, label_8]

In [3]:
density_matrix(state_labels[1])

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [4]:
X_tra = []
Y_ans = []
with open('winequality-red.csv', 'r') as f:
    x = f.readlines()
x=x[1:]
x
for i in range(len(x)):
    x[i] = x[i][:-1]
    x[i] = x[i].split(';')
    X_tra.append(x[i][:-1])
    Y_ans.append(x[i][-1:])
x
ans = []
for i in range(len(Y_ans)):
    if(Y_ans[i]==min(Y_ans)):
        Y_ans[i]=['0']
    ans.append(Y_ans[i][0])
for i in range(len(X_tra)):
    ans[i]=int(ans[i])
    for j in range(len(X_tra[i])):
        X_tra[i][j]=float(X_tra[i][j])

In [5]:
print(scaler.fit(X_tra))

MinMaxScaler(copy=True, feature_range=(0, 1))


In [6]:
X_tra = scaler.transform(X_tra)

In [7]:
X_tra

array([[0.24778761, 0.39726027, 0.        , ..., 0.60629921, 0.13772455,
        0.15384615],
       [0.28318584, 0.52054795, 0.        , ..., 0.36220472, 0.20958084,
        0.21538462],
       [0.28318584, 0.43835616, 0.04      , ..., 0.40944882, 0.19161677,
        0.21538462],
       ...,
       [0.15044248, 0.26712329, 0.13      , ..., 0.53543307, 0.25149701,
        0.4       ],
       [0.11504425, 0.35958904, 0.12      , ..., 0.65354331, 0.22754491,
        0.27692308],
       [0.12389381, 0.13013699, 0.47      , ..., 0.51181102, 0.19760479,
        0.4       ]])

In [8]:
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def qcircuit(params, x=None, y=None):
    for i in range(len(params)):
        qml.Rot(x[0]*(math.pi)/2,x[1]*(math.pi)/2,x[2]*(math.pi)/2, wires=0)
        qml.Rot(x[3]*(math.pi)/2,x[4]*(math.pi)/2,x[5]*(math.pi)/2, wires=0)
        qml.Rot(x[6]*(math.pi)/2,x[7]*(math.pi)/2,x[8]*(math.pi)/2, wires=0)
        qml.Rot(x[9]*(math.pi)/2,x[10]*(math.pi)/2,0, wires=0)
        qml.Rot(x[0]*(math.pi)/2,x[1]*(math.pi)/2,x[2]*(math.pi)/2, wires=1)
        qml.Rot(x[3]*(math.pi)/2,x[4]*(math.pi)/2,x[5]*(math.pi)/2, wires=1)
        qml.Rot(x[6]*(math.pi)/2,x[7]*(math.pi)/2,x[8]*(math.pi)/2, wires=1)
        qml.Rot(x[9]*(math.pi)/2,x[10]*(math.pi)/2,0, wires=1)
        qml.Rot(x[0]*(math.pi)/2,x[1]*(math.pi)/2,x[2]*(math.pi)/2, wires=2)
        qml.Rot(x[3]*(math.pi)/2,x[4]*(math.pi)/2,x[5]*(math.pi)/2, wires=2)
        qml.Rot(x[6]*(math.pi)/2,x[7]*(math.pi)/2,x[8]*(math.pi)/2, wires=2)
        qml.Rot(x[9]*(math.pi)/2,x[10]*(math.pi)/2,0, wires=2)
        qml.Rot(x[0]*(math.pi)/2,x[1]*(math.pi)/2,x[2]*(math.pi)/2, wires=3)
        qml.Rot(x[3]*(math.pi)/2,x[4]*(math.pi)/2,x[5]*(math.pi)/2, wires=3)
        qml.Rot(x[6]*(math.pi)/2,x[7]*(math.pi)/2,x[8]*(math.pi)/2, wires=3)
        qml.Rot(x[9]*(math.pi)/2,x[10]*(math.pi)/2,0, wires=3)
        qml.Rot(params[i][0]*(math.pi)/2,params[i][1]*(math.pi)/2,params[i][2]*(math.pi)/2, wires=0)
        qml.Rot(params[i][3]*(math.pi)/2,params[i][4]*(math.pi)/2,params[i][5]*(math.pi)/2, wires=0)
        qml.Rot(params[i][6]*(math.pi)/2,params[i][7]*(math.pi)/2,params[i][8]*(math.pi)/2, wires=0)
        qml.Rot(params[i][9]*(math.pi)/2,params[i][10]*(math.pi)/2,params[i][11]*(math.pi)/2, wires=0)
        qml.Rot(params[i][12]*(math.pi)/2,params[i][13]*(math.pi)/2,params[i][14]*(math.pi)/2, wires=1)
        qml.Rot(params[i][15]*(math.pi)/2,params[i][16]*(math.pi)/2,params[i][17]*(math.pi)/2, wires=1)
        qml.Rot(params[i][18]*(math.pi)/2,params[i][19]*(math.pi)/2,params[i][20]*(math.pi)/2, wires=1)
        qml.Rot(params[i][21]*(math.pi)/2,params[i][22]*(math.pi)/2,params[i][23]*(math.pi)/2, wires=1)
        qml.Rot(params[i][24]*(math.pi)/2,params[i][25]*(math.pi)/2,params[i][26]*(math.pi)/2, wires=2)
        qml.Rot(params[i][27]*(math.pi)/2,params[i][28]*(math.pi)/2,params[i][29]*(math.pi)/2, wires=2)
        qml.Rot(params[i][30]*(math.pi)/2,params[i][31]*(math.pi)/2,params[i][32]*(math.pi)/2, wires=2)
        qml.Rot(params[i][33]*(math.pi)/2,params[i][34]*(math.pi)/2,params[i][35]*(math.pi)/2, wires=2)
        qml.Rot(params[i][36]*(math.pi)/2,params[i][37]*(math.pi)/2,params[i][38]*(math.pi)/2, wires=3)
        qml.Rot(params[i][39]*(math.pi)/2,params[i][40]*(math.pi)/2,params[i][41]*(math.pi)/2, wires=3)
        qml.Rot(params[i][42]*(math.pi)/2,params[i][43]*(math.pi)/2,params[i][44]*(math.pi)/2, wires=3)
        qml.Rot(params[i][45]*(math.pi)/2,params[i][46]*(math.pi)/2,params[i][47]*(math.pi)/2, wires=3)
#        if i== len(params) -1 :
#           break
#        qml.Hadamard(wires=1)
#        qml.CNOT(wires=[0,1])
#       qml.Hadamard(wires=1)
    return qml.expval(qml.Hermitian(y, wires=[0,1,2,3]))


def fidelity(state1, state2):
    return np.abs(np.dot(np.conj(state1), state2))


def cost(params, x, y, state_labels=None):
    loss = 0.0
    dm_labels = [density_matrix(s) for s in state_labels]
    for i in range(len(x)):
        f = qcircuit(params, x=x[i], y=dm_labels[y[i]])
        loss = loss + (1 - f) ** 2
    return loss / len(x)

Utility functions for testing and creating batches
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



In [9]:
def test(params, x, y, state_labels=None):
    fidelity_values = []
    output_states = []
    dm_labels = [density_matrix(s) for s in state_labels]
    for i in range(len(x)):
        expectation = qcircuit(params, x=x[i], y=dm_labels[y[i]])
        output_states.append(dev._state)
    predicted = predicted_labels(output_states, state_labels)
    return predicted, output_states


def predicted_labels(states, state_labels=None):
    output_labels = [
        np.argmax([fidelity(s, label) for label in state_labels]) for s in states
    ]
    return np.array(output_labels)


def accuracy_score(y_true, y_pred):
    score = y_true == y_pred
    return score.sum() / len(y_true)


def iterate_minibatches(inputs, targets, batch_size):
    for start_idx in range(0, inputs.shape[0] - batch_size + 1, batch_size):
        idxs = slice(start_idx, start_idx + batch_size)
        yield inputs[idxs], targets[idxs]

In [10]:
Xdata, Xtest, y_train, y_test = train_test_split(X_tra, ans, test_size=0.33, random_state=42)
Xdata = np.array(Xdata)
Xtest = np.array(Xtest)

Train a quantum classifier on the circle dataset
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



In [ ]:
# Generate training and test data
num_training = 200
num_test = 2000

#Xdata, y_train = circle(num_training)
X_train = np.hstack((Xdata, np.zeros((Xdata.shape[0], 1))))

#Xtest, y_test = circle(num_test)
X_test = np.hstack((Xtest, np.zeros((Xtest.shape[0], 1))))


# Train using Adam optimizer and evaluate the classifier
num_layers = 2
learning_rate = 0.6
epochs = 5
batch_size = 32

opt = AdamOptimizer(learning_rate, beta1=0.9, beta2=0.999)

# initialize random weights
params = np.random.uniform(size=(num_layers, 48))
print(params)
predicted_train, states_train = test(params, X_train, y_train, state_labels)
accuracy_train = accuracy_score(y_train, predicted_train)

predicted_test, states_test = test(params, X_test, y_test, state_labels)
accuracy_test = accuracy_score(y_test, predicted_test)

# save predictions with random weights for comparison
initial_predictions = predicted_test

loss = cost(params, X_test, y_test, state_labels)

print(
    "Epoch: {:2d} | Cost: {:3f} | Train accuracy: {:3f} | Test Accuracy: {:3f}".format(
        0, loss, accuracy_train, accuracy_test))

for it in range(epochs):
    for Xbatch, ybatch in iterate_minibatches(X_train, y_train, batch_size=batch_size):
        params = opt.step(lambda v: cost(v, Xbatch, ybatch, state_labels), params)

    predicted_train, states_train = test(params, X_train, y_train, state_labels)
    accuracy_train = accuracy_score(y_train, predicted_train)
    loss = cost(params, X_train, y_train, state_labels)

    predicted_test, states_test = test(params, X_test, y_test, state_labels)
    accuracy_test = accuracy_score(y_test, predicted_test)
    res = [it + 1, loss, accuracy_train, accuracy_test]
    print(
        "Epoch: {:2d} | Loss: {:3f} | Train accuracy: {:3f} | Test accuracy: {:3f}".format(
            *res))

[[0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
  0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
  0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643
  0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184
  0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
  0.60754485 0.17052412 0.06505159 0.94888554 0.96563203 0.80839735
  0.30461377 0.09767211 0.68423303 0.44015249 0.12203823 0.49517691
  0.03438852 0.9093204  0.25877998 0.66252228 0.31171108 0.52006802]
 [0.54671028 0.18485446 0.96958463 0.77513282 0.93949894 0.89482735
  0.59789998 0.92187424 0.0884925  0.19598286 0.04522729 0.32533033
  0.38867729 0.27134903 0.82873751 0.35675333 0.28093451 0.54269608
  0.14092422 0.80219698 0.07455064 0.98688694 0.77224477 0.19871568
  0.00552212 0.81546143 0.70685734 0.72900717 0.77127035 0.07404465
  0.35846573 0.11586906 0.86310343 0.62329813 0.33089802 0.06355835
  0.31098232 0.32518332 0.72960618 0.63755747 0

Results
~~~~~~~



In [ ]:
#print("Cost: {:3f} | Train accuracy {:3f} | Test Accuracy : {:3f}".format(
 #       loss, accuracy_train, accuracy_test))

print("Learned weights")
for i in range(num_layers):
    print("Layer {}: {}".format(i, params[i]))


fig, axes = plt.subplots(1, 3, figsize=(10, 3))
plot_data(X_test, initial_predictions, fig, axes[0])
plot_data(X_test, predicted_test, fig, axes[1])
plot_data(X_test, y_test, fig, axes[2])
axes[0].set_title("Predictions with random weights")
axes[1].set_title("Predictions after training")
axes[2].set_title("True test data")
plt.show()

This tutorial was generated using the following Pennylane version:



In [ ]:
qml.about()

References
----------
[1] Pérez-Salinas, Adrián, et al. “Data re-uploading for a universal
quantum classifier.” arXiv preprint arXiv:1907.02085 (2019).

[2] Kingma, Diederik P., and Ba, J. "Adam: A method for stochastic
optimization." arXiv preprint arXiv:1412.6980 (2014).

[3] Liu, Dong C., and Nocedal, J. "On the limited memory BFGS
method for large scale optimization." Mathematical programming
45.1-3 (1989): 503-528.

